In [53]:
%reset

import torch
import funciones
import pandas as pd
import re
import multidict as multidict

import numpy as np

import os
import re
from PIL import Image
from os import path
from wordcloud import WordCloud
import matplotlib.pyplot as plt

import tweepy

from unicodedata import normalize

import time

from matplotlib.font_manager import FontProperties



Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [157]:
def scrapeo(hashtag,cantidad=1000,fecha='2017-09-10'):
    message,retweet_count,created_at,user_name,coord,geo,metadata=[],[],[],[],[],[],[]
    i = 0
    tweepy.Status
    for tweet in tweepy.Cursor(api.search,q="#"+str(hashtag),count = cantidad, lang = 'es', since = fecha,tweet_mode='extended').items():
        if 'RT' in tweet.full_text:
            continue
        message.append(tweet.full_text)
        retweet_count.append(tweet.retweet_count)
        created_at.append(tweet.created_at)
        user_name.append(tweet.user.name)
        coord.append(tweet.coordinates)
        geo.append(tweet.geo)
        metadata.append(tweet.metadata)
        i+=1
        if i%1000 == 0: print (i,tweet.full_text)
        if i == 10000: break
    print (len(message),len(retweet_count),len(created_at),len(user_name),len(coord),len(geo))
    df=pd.DataFrame({'Message':message,
                    'Retweet Count':retweet_count,
                    'Created At':created_at,
                    'Username':user_name,
                     'Geo':geo,
                     'Coord':coord,
                     'Metadata':metadata
                    })
    df.to_csv("Twitter_%s.csv"%(hashtag))
    return(df)

    

def diccionario (msj):
    #Patrones a filtrar
    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    palabras_borradas = """ so|o|su|vos|re|pq|le|te|tu|soy|sos|mi|da|n|c|m|o|u|x|s|les|me|d|q|1|2|3|4|5|6|7|8|9|0|e|como|lo|los|mi|
    son|donde|a|el|un|la|una|en|por|para|ante|si|sobre|sin|al|rt|del|y|se|de|que|sus|ha|es|con|esta|ni|no """
    palabras_borradas= " | ".join(palabras_borradas.split('|'))
    #Hace el word to ix
    word_to_ix = {}
    word_cant = {}
    emoji = []
    emoji_ix = {}
    emoji_cant = {}
    tw_corr = []
    for i,twit in enumerate(msj):
        if i %1000 == 0: print(i, twit)
        txt = pattern.sub('',twit)
        txt = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1",normalize( "NFD", txt), 0, re.I)
        txt = normalize( 'NFC', txt).lower()
        txt = txt.replace('\n',' ').replace(';','').replace('°','').replace('ñ','n').replace('\ufe0f','').replace('+','').replace('¡','').replace('¿','').replace('?','').replace('▯','').replace('(','').replace('|','').replace('!','').replace(')','').replace('.','').replace('-','').replace('_','').replace(',','').replace('#','').replace(':',' ')
        tt = txt
        for i in txt:
            if str(i).isascii()==False:
                tt = tt.replace(i,' '+i.encode('unicode-escape').decode('ASCII')+' ')
                emoji.append(i)
        txt = tt
                
        nlist = txt.lower().split(' ')

        for word in nlist:
            
            
            if re.match(palabras_borradas, word):
                continue
            if ('@' in word) and ('RELAMPAGO2018' not in word) and ('RELAMPAGO_edu' not in word) or ('jaj' in word):
                continue
            
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
                word_cant[word]  = 1
            else: word_cant[word]+=1
        for emo in emoji:
            if emo not in emoji_ix:
                emoji_ix[emo] = len(emoji_ix)
                emoji_cant[emo]  = 1
            else: emoji_cant[emo]+=1
                
        txt = " ".join(nlist) 
        tw_corr.append(txt)

    vocab_size = len(word_to_ix)
    num_labels = 2
    #print(emoji)
    
    
    
    return(tw_corr,word_to_ix,word_cant,emoji_ix,emoji_cant)

def makeImage(text):
    alice_mask = np.array(Image.open("cord.jpeg"))

    wc = WordCloud(background_color="white", max_words=200, mask=alice_mask)
    # generate word cloud
    wc.generate_from_frequencies(text)

    # show
    fig = plt.figure(figsize=(20,20),dpi=100)
    plt.imshow(wc, interpolation="bilinear",cmap=plt.get_cmap("coolwarm"))
    #plt.axis("off")
    plt.tick_params(bottom=False,top=False,left=False,right=False,labelbottom=False,labelleft = False)
    #plt.show()
    plt.savefig('Multiword')

def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence.split():
        if word not in word_to_ix: continue
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)

In [63]:
tw = pd.read_csv('tweets.csv',sep = ';')
msj = tw['text']

## ESTO NO SE TOCA

In [158]:
d = diccionario(msj)


0 ¡Ni la lluvia los detuvo!🌧️ Este mañana se corrieron los #10KTernium con gran participación de corredores nicoleños. 🏅

🏃‍♂️Caballeros: 1° Ulises Otero, 2° Juan Ansio, 3° Mario Raschetti 
🏃‍♀️Damas: 1° Ludmila Martinez, 2° Nerea Mendoza, 3° María de los Ángeles Troffe https://t.co/eZceEaGQO7
1000 Que hermoso despertar con lluvia 💓
2000 @gabicerru Que6 raro que no te acuerdes de la inundación de La Plata, gobernación Scioli e Intendencia Bruera...más de 200 muertos y perdidas de $2618millones...que memoria selectiva! https://t.co/SfF7DUWmQm
3000 Llueve???
4000 A mi me gustaba la lluvia. Hasta hoy
5000 *habia tormenta electrica*
6000 @CamiPuy1 Dejame tranquilooooo, ni con la inundacion me das respiro, me asfixias
7000 Se deja de tenerle miedo a las tormentas? https://t.co/mPpmmQxNmT
8000 Ya con el sol, para mí era un día gris. La lluvia hoy no me ayuda..
9000 Que ganas de cagarla la lluvia😑
10000 No tenía ni ganas de viajar, gracias lluvia😝
11000 Lluvia todo el finde jaja que bajon par

## ENSERIO

In [103]:
#Hago el bago of words

bow = []

for i,t in enumerate(msj):   
    bow.append(make_bow_vector(t,d[1]))
    if i % 1000==0:print(i,bow[-1])
bow

0 tensor([[1., 0., 0.,  ..., 0., 0., 0.]])
1000 tensor([[1., 0., 0.,  ..., 0., 0., 0.]])
2000 tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
3000 tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
4000 tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
5000 tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
6000 tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
7000 tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
8000 tensor([[1., 0., 0.,  ..., 0., 0., 0.]])
9000 tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
10000 tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
11000 tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
12000 tensor([[1., 0., 0.,  ..., 0., 0., 0.]])
13000 tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
14000 tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
15000 tensor([[1., 0., 0.,  ..., 0., 0., 0.]])
16000 tensor([[1., 0., 0.,  ..., 0., 0., 0.]])
17000 tensor([[1., 0., 0.,  ..., 0., 0., 0.]])
18000 tensor([[1., 0., 0.,  ..., 0., 0., 0.]])
19000 tensor([[1., 0., 0.,  ..., 0., 0., 0.]])
20000 tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
21000 tensor([[0., 0., 0.,

[tensor([[1., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0

In [130]:
dd  =list(d[1].keys())
dd = np.asarray(dd)
#print(np.dot(dd,bow[0][0,:]))